In [1]:
import pandas as pd

from src.metrics import map_score, mrr_score, ndcg_score, rmse_score
from src.models.funk_svd import FunkSVDRecommender
from src.utils import train_test_split, to_user_movie_matrix, make_binary_matrix, RatingMatrix

Let's load the datasets with users info, movies info and users' ratings for movies.

Then we split it to training/test subsets by the timestamp.

In [2]:
users = pd.read_table("../data/users.dat", sep="::", names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], engine='python')

movies = pd.read_table("../data/movies.dat", sep="::", names=['MovieID', 'Title', 'Genres'], engine='python', encoding='latin1')

ratings = pd.read_table("../data/ratings.dat", sep="::", names=['UserID', 'MovieID', 'Rating', 'Timestamp'], engine='python')
ratings['Timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')

ratings = ratings[ratings['MovieID'].isin(movies['MovieID'])]

train_ratings, test_ratings = train_test_split(ratings, 'Timestamp')
user_movie_train = to_user_movie_matrix(train_ratings)
user_movie_test = to_user_movie_matrix(test_ratings)

Now, we are going to train our Funk SVD recommender model. It depends on two parameters factor and alpha, so we will do the cross validation to find optimal values based on 4 metrics:
* mean average precision (MAP)
* mean reciprocal rank (MRR)
* normalized discounted cumulative gain (NDCG)
* root mean squared error (RMSE)

In [3]:
import numpy as np
from tqdm import tqdm

factors = [1,2,4,8]
alphas = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
lambdas = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]

In [4]:
map_list = list()
mrr_list = list()
ndcg_list = list()
rmse_list = list()
for factor in tqdm(factors):
    for alpha in tqdm(alphas):
        for lambda_ in tqdm(lambdas):
            model = FunkSVDRecommender()
            model.train(user_movie_train, factor, alpha=alpha, lambda_=alpha)
            
            y_pred = model.predict(make_binary_matrix(user_movie_test.get_rating_matrix()))
            map_list.append({"factor": factor, "lambda": alpha, "map": map_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred, top=10)})
            mrr_list.append({"factor": factor, "lambda": alpha, "mrr": mrr_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred, top=10)})
            ndcg_list.append({"factor": factor, "lambda": alpha, "ndcg": ndcg_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred, top=10)})
            rmse_list.append({"factor": factor, "lambda": alpha, "rmse": rmse_score(RatingMatrix(user_movie_test.get_rating_matrix()[y_pred.get_users()]), y_pred)})

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:15<01:35, 15.90s/it]

 29%|██▊       | 2/7 [00:31<01:19, 15.97s/it]

 43%|████▎     | 3/7 [00:48<01:04, 16.25s/it]

 57%|█████▋    | 4/7 [01:05<00:49, 16.43s/it]

 71%|███████▏  | 5/7 [01:25<00:35, 17.73s/it]

 86%|████████▌ | 6/7 [01:45<00:18, 18.69s/it]

100%|██████████| 7/7 [02:04<00:00, 17.85s/it]

 14%|█▍        | 1/7 [02:04<12:29, 124.96s/it]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:16<01:37, 16.31s/it]

 29%|██▊       | 2/7 [00:32<01:22, 16.48s/it]

 43%|████▎     | 3/7 [00:49<01:05, 16.48s/it]

 57%|█████▋    | 4/7 [01:06<00:50, 16.84s/it]

 71%|███████▏  | 5/7 [01:24<00:34, 17.03s/it]

 86%|████████▌ | 6/7 [01:41<00:17, 17.16s/it]

100%|██████████| 7/7 [01:58<00:00, 16.95s/it]

 29%|██▊       | 2/7 [04:03<10:06, 121.26s/it]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:18<01:50, 18.37s/it]

 29%|██▊       | 2/7 [00:36<01:31, 18.22s/i

In [5]:
map_df = pd.DataFrame(map_list)
mrr_df = pd.DataFrame(mrr_list)
ndcg_df = pd.DataFrame(ndcg_list)
rmse_df = pd.DataFrame(rmse_list)

In [6]:
map_df[map_df['map'] == map_df['map'].max()]

,factor,lambda,map
151,8,0.1,0.264479


In [7]:
mrr_df[mrr_df['mrr'] == mrr_df['mrr'].max()]

,factor,lambda,mrr
151,8,0.1,0.593452


In [8]:
ndcg_df[ndcg_df['ndcg'] == ndcg_df['ndcg'].max()]

,factor,lambda,ndcg
151,8,0.1,0.361934


In [9]:
rmse_df[rmse_df['rmse'] == rmse_df['rmse'].min()]

,factor,lambda,rmse
151,8,0.1,1.544805


The best possible MAP is 0.264 indicates that, on average, about 26.4% of the top-10 recommended items are relevant.

The best possible MRR is 0.593 means that, on average, the first relevant item appears between the 1st and 2nd positions in the recommendation list.

NDCG measures the quality of the recommendations by considering the position of the relevant items in the list, with higher-ranked items contributing more to the score. The best possible NDCG is 0.361.

RMSE measures the differences between the predicted and actual ratings. The best possible RMSE is 1.544.